# Regression Beyond Food

While we were able to fit models relating various measures of risk of death from non-communicable diseases, we would expect other variables to also play some role in the variation not captured by those relationships we found. One approach we could use to determine whether other features i.e. exercise, access to doctors could also be important would be to use the residuals of the models we fitted as the responses for new linear regression models with the non-food variables as features.

In [1]:
# importing modules
import pickle
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import sklearn as sk
from sklearn import linear_model
%matplotlib inline

First we need to load in both the new data and the models we calculated earlier.

In [13]:
# loading in the data
out = open('data/final/beds_1970_2000.p', 'r')
beds_1970_2000 = pickle.load(out)
out.close()
out = open('data/final/doctors_1970_2000.p', 'r')
doctors_1970_2000 = pickle.load(out)
out.close()
out = open('data/clean/exer.p', 'r')
exercise = pickle.load(out)
out.close()
# also loading in the index for the countries we need to drop
out = open('')

In [14]:
exercise

,Percent
Country,
Afghanistan,NaN
Albania,NaN
Algeria,34.4
Angola,NaN
Argentina,39.2
Armenia,NaN
Australia,23.8
Austria,23.8
Azerbaijan,NaN


In [3]:
# loading in the lasso models we selected from the scoring notebook
out = open('data/models/risk_results_lasso.p', 'r')
risk_results_lasso = pickle.load(out)
out.close()
out = open('data/models/deaths_all_results_lasso.p', 'r')
deaths_all_results_lasso = pickle.load(out)
out.close()
out = open('data/models/deaths_cancer_results_lasso.p', 'r')
deaths_cancer_results_lasso = pickle.load(out)
out.close()
out = open('data/models/deaths_cardio_results_lasso.p', 'r')
deaths_cardio_results_lasso = pickle.load(out)
out.close()
out = open('data/models/deaths_diabetes_results_lasso.p', 'r')
deaths_diabetes_results_lasso = pickle.load(out)
out.close()
out = open('data/models/deaths_resp_results_lasso.p', 'r')
deaths_resp_results_lasso = pickle.load(out)
out.close()

Now we need to collect the residuals of each model so we can use them as responses.

In [11]:
# collecting the residuals from each model
risk_lasso_resid = risk_results_lasso.resid
deaths_all_lasso_resid = deaths_all_results_lasso.resid
deaths_cancer_lasso_resid = deaths_cancer_results_lasso.resid
deaths_cardio_lasso_resid = deaths_cardio_results_lasso.resid
deaths_diabetes_lasso_resid = deaths_diabetes_results_lasso.resid
deaths_resp_lasso_resid = deaths_resp_results_lasso.resid

And now we can fit models with each of these residual datasets as the responses and the doctors data from 1970 to 2000 as the feature.

In [ ]:
risk_exercise